# FastText

## Import et préparation des données

On implémente le modèle "FastText" issu de l'article <a href="https://arxiv.org/pdf/1607.01759.pdf">Bag of Tricks for Efficient Text Classification.</a> 



On définit une fonction qui génère des bigrames comme dans l'exemple ci-dessous.

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

generate_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'film is', 'This film', 'is terrible']

On pré-traite les données comme précédemment en utilisant la fonction generate_bigrams comme paramètre de la méthode `Field`.

In [2]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')


Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


In [3]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,sort=False)

## Construire le modèle

Ce modèle a beaucoup moins de paramètres que le modèle précédent car il n'a que 2 couches qui ont des paramètres, la couche d'intégration et la couche linéaire. Il n'y a pas de composant RNN.

<center> <img src="fast.png" alt="drawing" width="300"/>
    
Au lieu de cela, il calcule d'abord le mot incorporé pour chaque mot à l'aide de l'embedding, puis calcule la moyenne de tous les mots incorporés et le transmet à travers la couche linéaire.

Nous implémentons le calcul de moyenne avec la fonction avg_pool2d. En effet, on peut considérer les mots incorporés comme une grille à 2 dimensions, où les mots sont le long d'un axe et les dimensions des mots incorporés le long de l'autre.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)
    
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner')

Le modèle a 2,500,301 paramètres à entraîner


Ce modèle a deux fois moins de paramètres.

In [6]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [7]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [8]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


def recall(preds, y):
    '''
    Retourne le recall
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float()       
    
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    if (tp + fn) == 0:
        recall = torch.zeros(1)
        
    recall = tp / (tp + fn)
    return recall



def f1_loss(preds, y):
    '''
    Retourne le score F1
    '''  
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    if (tp + fn) == 0 or (tp + fp) == 0 or (recall + precision == 0):
        f1 = torch.zeros(1)
    else:
        f1 = 2* (precision*recall) / (precision + recall)
    
    return f1

In [9]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        rec = recall(predictions, batch.label)
        f1 = f1_loss(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_rec += rec.item()
        epoch_f1 += f1.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            rec = recall(predictions, batch.label)
            f1 = f1_loss(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_rec += rec.item()
            epoch_f1 += f1.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),epoch_rec / len(iterator), epoch_f1 / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc,  train_rec, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc,  valid_rec, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%| Train Recall: {train_rec*100:.2f}%| Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%| Val. Recall: {valid_rec*100:.2f}%| Val. F1: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 21s
	Train Loss: 0.652 | Train Acc: 63.91%| Train Recall: 15.19%| Train F1: 24.05%
	 Val. Loss: 0.625 |  Val. Acc: 72.41%| Val. Recall: 31.74%| Val. F1: 45.82%
Epoch: 02 | Epoch Time: 0m 22s
	Train Loss: 0.586 | Train Acc: 78.17%| Train Recall: 35.56%| Train F1: 49.85%
	 Val. Loss: 0.555 |  Val. Acc: 79.91%| Val. Recall: 41.22%| Val. F1: 55.15%
Epoch: 03 | Epoch Time: 0m 22s
	Train Loss: 0.512 | Train Acc: 83.38%| Train Recall: 40.71%| Train F1: 55.08%
	 Val. Loss: 0.494 |  Val. Acc: 82.22%| Val. Recall: 41.88%| Val. F1: 56.30%
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.447 | Train Acc: 85.95%| Train Recall: 41.78%| Train F1: 56.51%
	 Val. Loss: 0.444 |  Val. Acc: 84.39%| Val. Recall: 44.01%| Val. F1: 58.20%
Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 0.396 | Train Acc: 87.88%| Train Recall: 42.55%| Train F1: 57.49%
	 Val. Loss: 0.407 |  Val. Acc: 85.61%| Val. Recall: 44.05%| Val. F1: 58.59%
Epoch: 06 | Epoch Time: 0m 22s
	Train Loss: 0.355 | Train Acc: 89

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc, test_rec, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%| Test Recall: {test_rec*100:.2f}%| Test F1: {test_f1*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")

## Référence

 - https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb